In [ ]:
import finnhub
import pandas as pd
import datetime
import requests
from bs4 import BeautifulSoup
import re
import sqlite3
import time
from sqlite3 import OperationalError

In [56]:
class trade_idea:
    def __init__(self, symbol, time_frame, position, time_posted, user):
        self.symbol = symbol
        self.time_frame = time_frame
        self.position = position
        self.time_posted = time_posted
        self.user = user

def get_trade_ideas(begin_page, end_page):
    trade_ideas = []
    for i in range(begin_page, end_page):
        trade_ideas.extend(store_all_data("https://www.tradingview.com/markets/stocks-usa/ideas/page-%d/" % i))
    print(len(trade_ideas))
    return trade_ideas

def store_all_data(URL):
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    data = soup.find_all("div", class_="tv-widget-idea js-userlink-popup-anchor")
    trade_ideas = []
    print(len(data))
    for i in data:
        symbol = i.find('a', class_="tv-widget-idea__symbol apply-overflow-tooltip")
        time_frame = i.find_all('span', class_="tv-widget-idea__timeframe")[1]
        position = i.find('span', class_="content-yHuWj4ze badge-idea-content-Naj79Gc7")
        time_posted = i.find('span', class_="tv-card-stats__time")
        user = i.find('span', class_="tv-card-user-info__name")
        if(symbol != None and time_frame != None and position != None and time_posted != None and user != None):
            trade_ideas.append(trade_idea(symbol.text, time_frame.text, position.text, time_posted['data-timestamp'], user.text))          
    return trade_ideas

def calculate_profit(stock, profit_exit, loss_exit):
    buy = 100
    initial_price = stock['Open'][0]
    exit1 = profit_exit
    exit2 = loss_exit
    for i in stock['Open']:
        if i / initial_price > exit1:
            print("initial_price: ", initial_price, "current price: ", i)
            return (buy * (i / initial_price)) - buy
        if i / initial_price < exit2:
            print("initial_price: ", initial_price, "current price: ", i)
            return (buy * (i / initial_price)) - buy
    return 0

def calculate_total_long_profit(trade_ideas, profit_exit_margin, exit_trade_margin):
    sum = 0
    for i in trade_ideas:
        try:
            stock = get_stock_db(i.symbol, i.time_posted, time.time())
        except OperationalError:
            stock = get_stock_api(i.symbol, i.time_posted, time.time())
        if i.position == "Long":
            profit = calculate_profit(stock, 1+profit_exit_margin, 1-exit_trade_margin)
            sum += profit
            print(i.symbol, i.time_frame, i.position, profit)
            
        elif i.position == "Short":
            profit = -calculate_profit(stock, 1+exit_trade_margin, 1-profit_exit_margin)
            sum += profit
            print(i.symbol, i.time_frame, i.position, profit)
    print("TOTAL PROFIT: ", sum)
    
def get_stock_db(stock_symbol, date_begin, date_end):
    con = sqlite3.connect('TradingBotDatabase.db')
    cur = con.cursor()
    date_begin = datetime.datetime.fromtimestamp(int(float(date_begin))).strftime("%Y-%m-%d %H:%M:%S")
    date_end = datetime.datetime.fromtimestamp(int(float(date_end))).strftime("%Y-%m-%d %H:%M:%S")
    print(stock_symbol, date_begin, date_end)
    cur.execute(f"""SELECT *     
        FROM {stock_symbol} 
        WHERE timestamp > '{date_begin}'
        AND timestamp < '{date_end}';""")
    stock = pd.DataFrame(cur.fetchall()).set_index(0)
    stock.rename(columns={0: 'Timestamp', 1: 'Open', 2: 'High', 3:'Low',4:'Close',5:'Volume',6:'Trade Count',7:'VWAP'}, inplace=True)
    print('')
    return stock

def get_stock_api(stock_symbol, date_begin, date_end):
    data = finnhub_client.stock_candles(stock_symbol, 'D', int(float(date_begin)), int(float(date_end)))
    df = pd.DataFrame(data)
    df['t'] = df['t'].apply(lambda x: datetime.datetime.fromtimestamp(x))
    stock = df[['t','o','h','l','c','v']].set_index('t')
    stock.rename(columns={'o': 'Open', 'h': 'High', 'l':'Low','c':'Close','v':'Volume'}, inplace=True)
    return stock        

def print_trade_ideas():
    for i in trade_ideas:
        print(i.symbol, i.time_frame, i.position, i.time_posted, i.user)


In [ ]:
finnhub_client = finnhub.Client(api_key="cb40dtaad3i8tak19g00")

In [ ]:
print_trade_ideas()

In [ ]:
get_stock_api("TSLA", 1656585469, 1657635469)

In [ ]:
get_stock_db("TSLA", 1656585469, 1657635469)

In [ ]:
trade_ideas = get_trade_ideas(148,151)

In [57]:
calculate_total_long_profit(trade_ideas, 0.010, 0.005)

WIT 2022-07-22 15:33:53 2022-08-03 17:04:56
initial_price:  5.15 current price:  5.04
WIT 1W Long -2.135922330097088
META 2022-07-22 15:28:39 2022-08-03 17:04:56
initial_price:  171.11 current price:  170.15
META 30 Long -0.561042604172755
CDMO 2022-07-22 14:49:20 2022-08-03 17:04:56
initial_price:  19.66 current price:  17.53
CDMO 1D Long -10.834181078331625
SOPH 2022-07-22 14:40:38 2022-08-03 17:04:56
initial_price:  3.89 current price:  3.57
SOPH 1D Long -8.226221079691527
CAL 2022-07-22 14:38:15 2022-08-03 17:04:56
initial_price:  27.85 current price:  27.33
CAL 1D Long -1.8671454219030608
ZM 2022-07-22 13:16:23 2022-08-03 17:04:56
initial_price:  111.0 current price:  106.39
ZM 240 Long -4.153153153153156
AAPL 2022-07-22 11:51:46 2022-08-03 17:04:56
initial_price:  155.7 current price:  154.1099
AAPL 1D Short 1.0212588310854045
TSLA 2022-07-22 11:37:00 2022-08-03 17:04:56
initial_price:  819.4 current price:  828.85
TSLA 480 Long 1.1532828899194527
SPY 2022-07-22 11:36:20 2022-08-

In [ ]:
stock = get_stock("SPY", 1657344370.0, time.time())

In [ ]:
original_price = stock["Open"][0]

In [ ]:
original_price

In [ ]:
stock['% change'] = stock.apply(lambda x: 1 - (x['Open'] / original_price), axis = 1)

In [ ]:
stock